In [2]:
import pandas, os
import arcpy

In [15]:
# in the input_diretory assign NAD1983 projection and outputs in a same directory
input_directory = "D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\Coastalwetland\\shapefiles\\UTM16_new"
default_source_sr = arcpy.SpatialReference(26916)  # NAD83 / UTM zone 16N (assumed source)
for filename in os.listdir(input_directory):
    if filename.endswith('.shp') and 'NAD1983' not in filename:
        shapefile_path = os.path.join(input_directory, filename)
        output_shapefile_name = os.path.splitext(filename)[0] + '_NAD1983.shp'
        output_shapefile_path = os.path.join(input_directory, output_shapefile_name)
        
        # Define the spatial reference for NAD 1983 (target)
        nad1983_sr = arcpy.SpatialReference(4269)  # EPSG code for NAD 1983 geographic
        
        # Get or set the source spatial reference
        src_sr = arcpy.Describe(shapefile_path).spatialReference
        if not src_sr or src_sr.name == "Unknown":
            # Assume source is NAD83 / UTM zone 16N if undefined
            arcpy.DefineProjection_management(shapefile_path, default_source_sr)
            src_sr = default_source_sr
        
        # Project the shapefile to NAD 1983
        arcpy.Project_management(shapefile_path, output_shapefile_path, nad1983_sr, in_coor_system=src_sr)


In [22]:
# merge category shapefiles (avg, high, low, surge) from both directories into merged outputs
base_directory = "D:\\Users\\abolmaal\\Arcgis\\NASAOceanProject\\GIS_layer\\Coastalwetland\\shapefiles"
utm_directory = os.path.join(base_directory, "UTM16_new")
merged_directory = os.path.join(base_directory, "merged")

# Ensure merged directory exists
os.makedirs(merged_directory, exist_ok=True)

categories = ['ave', 'avg', 'high', 'low', 'surge']  # handle both ave/avg
shapefiles_to_merge = {cat: [] for cat in categories}

# Collect NAD1983 shapefiles from both locations
search_paths = [base_directory, utm_directory]
for search_path in search_paths:
    for filename in os.listdir(search_path):
        if filename.endswith('_NAD1983.shp'):
            full_path = os.path.join(search_path, filename)
            for cat in categories:
                if cat in filename:
                    shapefiles_to_merge[cat].append(full_path)
                    break

# Merge per category into merged_directory, preserving all fields via explicit field mappings
for cat, files in shapefiles_to_merge.items():
    if not files:
        print(f"No shapefiles found for category '{cat}'.")
        continue

    output_shapefile_path = os.path.join(merged_directory, f"Wetland_connected_{cat}_inundation_NAD1983.shp")
    if arcpy.Exists(output_shapefile_path):
        arcpy.Delete_management(output_shapefile_path)

    # Build field mappings to keep every attribute across inputs
    field_mappings = arcpy.FieldMappings()
    field_maps = {}
    for shp in files:
        for fld in arcpy.ListFields(shp):
            if fld.type in ("Geometry", "OID"):
                continue
            name = fld.name
            if name not in field_maps:
                fm = arcpy.FieldMap()
                fm.addInputField(shp, name)
                out_field = fm.outputField
                out_field.name = name
                out_field.aliasName = name
                fm.outputField = out_field
                field_maps[name] = fm
            else:
                field_maps[name].addInputField(shp, name)
    for fm in field_maps.values():
        field_mappings.addFieldMap(fm)

    arcpy.Merge_management(files, output_shapefile_path, field_mappings)
    print(f"Merged {len(files)} shapefiles for '{cat}' into {output_shapefile_path} with {len(field_maps)} fields preserved")

No shapefiles found for category 'ave'.
Merged 2 shapefiles for 'avg' into D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles\merged\Wetland_connected_avg_inundation_NAD1983.shp with 7 fields preserved
Merged 2 shapefiles for 'avg' into D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles\merged\Wetland_connected_avg_inundation_NAD1983.shp with 7 fields preserved
Merged 2 shapefiles for 'high' into D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles\merged\Wetland_connected_high_inundation_NAD1983.shp with 7 fields preserved
Merged 2 shapefiles for 'high' into D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles\merged\Wetland_connected_high_inundation_NAD1983.shp with 7 fields preserved
Merged 2 shapefiles for 'low' into D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles\merged\Wetland_connected_low_inundation_NAD1983.shp with 6 fields preserved
Merged 2 sh

In [21]:
# rename merged shapefiles to 'Wetland_connected_[cat]_inundation_NAD1983.shp'
for filename in os.listdir(merged_directory):
    if filename.startswith('merged_') and filename.endswith('.shp'):
        parts = filename.split('_')
        if len(parts) >= 3:
            cat = parts[1]
            new_filename = f'Wetland_connected_{cat}_inundation_NAD1983.shp'
            old_path = os.path.join(merged_directory, filename)
            new_path = os.path.join(merged_directory, new_filename)
            if arcpy.Exists(new_path):
                arcpy.Delete_management(new_path)
            arcpy.Rename_management(old_path, new_path, "FeatureClass")
            print(f"Renamed {filename} to {new_filename}")